# Modelos probabilísticos (Ejercicio)

## Aplicación de Naive Bayes multinomial a la detección de SMS *spam*

En este ejercicio se pide reproducir lo realizado en el caso práctico que se ha descrito en los vídeos (análisis de sentimiento en críticas de cine), pero ahora para detectar cuándo un mensaje corto (SMS) es *spam*.

### El conjunto de datos

El conjunto de datos consiste una serie de mensajes SMS (5574 en total), que están clasificados como mensajes basura (*spam*) o mensajes normales (*ham*). Los datos se pueden obtener en el [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). 

En concreto, descargar el fichero [smsspamcollection.zip](https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip), y descomprimirlo para obtener un fichero de texto SMSSpamCollection. En este fichero de texto hay una línea por cada sms, con el formato: *clase* *tabulador* *sms*. Por ejemplo, la primera línea es:

`ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...`

El fichero debe ser leído convenientemente para poder aplicar la vectorización. Se puede hacer la lectura usando las funciones python de lectura de ficheros, pero se recomienda usar la instrucción `read_table` de la biblioteca `pandas`:

In [1]:
import pandas as pd

*Pandas* es una biblioteca de python muy utilizada para manipular y analizar datos. Si el fichero se lee con la orden `read_table` (se pide averiguar la manera concreta de hacerlo), entonces se obtendrá una tabla (o *Data Frame*), en el que las etiquetas serán una columna y los correspondientes sms otra. Esto permite obtener de manera sencilla la lista de etiquetas o clases, y por otro lado la lista de mensajes, en el mismo orden.  

### Aprendiendo a clasificar SMSs

Se pide reproducir con estos datos lo realizado en el *notebook* en el que se aplica Naive Bayes Multinomial al análisis de sentimientos de críticas de cine, pero ahora para clasificar un SMS como *spam* o como normal. Esto incluye:

* Separación de los textos en entrenamiento y prueba 
* Vectorización de los textos 
* Aprendizaje con `MultinomialNB`
* Mostrar algunas clasificaciones sobre sms concretos.
* Rendimiento sobre entrenamiento y prueba.
* Ajuste manual del parámetro de suavizado
* Vectorización con `min_df` y `stop_words` 

**Nota**: este conjunto de datos no es balanceado (la mayoría son *ham*). Por tanto, usar `score` no es muy ilustrativo del rendimiento, ya que un clasificador "tonto" que siempre predijera *ham* tendría un rendimiento alto. Por ello, en este caso también se hace necesario usar el método `confusion_matrix` del módulo `metrics`. Se pide también explicar la salida que proporciona dicha métrica.

Se pide **comentar adecuadamente cada paso realizado**, relacionándolo con lo visto en la teoría. En particular, se pide mostrar parte de los atributos `class_count_`, `class_log_prior_`, `feature_count_` y `feature_log_prob_`, explicando claramente qué son cada uno de ellos. Explicar también cómo realiza las predicciones el modelo aprendido, tal y como se ha explicado en la teoría.  



In [32]:
# Pandas
convertTag = lambda x: 1 if x == 'ham' else 0
collection = pd.read_table('smsspamcollection/SMSSpamCollection', header=None, names = ['tag','text'], converters={'tag': convertTag } )
target_names = ['spam', 'ham']
print(collection)
# ¿quitar los '...'?

      tag                                               text
0       1  Go until jurong point, crazy.. Available only ...
1       1                      Ok lar... Joking wif u oni...
2       0  Free entry in 2 a wkly comp to win FA Cup fina...
3       1  U dun say so early hor... U c already then say...
4       1  Nah I don't think he goes to usf, he lives aro...
...   ...                                                ...
5567    0  This is the 2nd time we have tried 2 contact u...
5568    1               Will ü b going to esplanade fr home?
5569    1  Pity, * was in mood for that. So...any other s...
5570    1  The guy did some bitching but I acted like i'd...
5571    1                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [59]:
# Separacion de los textos en entrenamiento y prueba
messages = collection.text.tolist()
tags = collection.tag.tolist() #y_train

from sklearn.model_selection import train_test_split
text_train, text_test, y_train, y_test = train_test_split(messages, tags, stratify = tags, test_size = 0.2)

import numpy as np
print("Ejemplos por cada clase: {}".format(np.bincount(tags)))

Ejemplos por cada clase: [ 747 4825]


In [67]:
# Vectorizacion de los textos
print(type(text_train))
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(messages)
# print("Tamaño del vocabulario: {}".format(len(vect.vocabulary_)))
# print("Vocabulario:\n {}".format(vect.vocabulary_))

X_train = vect.transform(text_train)
#print("X_train:\n{}".format(repr(X_train)))
X_test = vect.transform(text_test)

<class 'list'>
X_train:
<4457x8713 sparse matrix of type '<class 'numpy.int64'>'
	with 59354 stored elements in Compressed Sparse Row format>
X_test:
<1115x8713 sparse matrix of type '<class 'numpy.int64'>'
	with 14815 stored elements in Compressed Sparse Row format>


In [69]:
# Aprendizaje con `MultinomialNB`
from sklearn.naive_bayes import MultinomialNB
multinb=MultinomialNB().fit(X_train,y_train)

In [71]:
print(multinb.class_count_)
print(multinb.class_log_prior_)
print(multinb.feature_count_)
print(multinb.feature_log_prob_)

[ 598. 3859.]
[-2.00864042 -0.14406781]
[[ 9. 22.  0. ...  0.  1.  0.]
 [ 0.  0.  0. ...  1.  0.  1.]]
[[ -7.73048231  -6.89757318 -10.0330674  ... -10.0330674   -9.33992022
  -10.0330674 ]
 [-10.98559776 -10.98559776 -10.98559776 ... -10.29245058 -10.98559776
  -10.29245058]]


In [89]:
# Clasificaciones de sms concretos
print("Octavo mensaje del conjunto de test: \n\n{}\n".format(text_test[7]))
print("Clasificación verdadera: {}.\n\n".format(y_test[7]))

print("Décimo mensaje del conjunto de test: \n\n{}\n".format(text_test[9]))
print("Clasificación verdadera: {}".format(y_test[9]))

print("Predicción del clasificador para el noveno mensaje: {}\n".format(multinb.predict(vect.transform([text_test[7]]))[0]))
print("Predicción del clasificador para el décimo mensaje: {}".format(multinb.predict(vect.transform([text_test[9]]))[0]))

print("Predicción de probabilidad para el noveno mensaje: {}\n".format(multinb.predict_proba(vect.transform([text_test[7]]))[0]))
print("Predicción de probabilidad para el décimo mensaje: {}".format(multinb.predict_proba(vect.transform([text_test[9]]))[0]))

Octavo mensaje del conjunto de test: 

It took Mr owl 3 licks

Clasificación verdadera: 1.


Décimo mensaje del conjunto de test: 

CALL 09090900040 & LISTEN TO EXTREME DIRTY LIVE CHAT GOING ON IN THE OFFICE RIGHT NOW TOTAL PRIVACY NO ONE KNOWS YOUR [sic] LISTENING 60P MIN 24/7MP 0870753331018+

Clasificación verdadera: 0
Predicción del clasificador para el noveno mensaje: 1

Predicción del clasificador para el décimo mensaje: 1
Predicción de probabilidad para el noveno mensaje: [0.02587709 0.97412291]

Predicción de probabilidad para el décimo mensaje: [0.43332185 0.56667815]


In [102]:
# Rendimiento sobre entrenamiento y prueba.
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

predictTrain = multinb.predict(X_train)
predictTest = multinb.predict(X_test)
from sklearn.metrics import confusion_matrix
matrixTrain = confusion_matrix(y_train, predictTrain)
spamsTrain = matrixTrain[0][0]/matrixTrain[0][0]+matrixTrain[1][0]
hamsTrain = matrixTrain[1][1]/matrixTrain[1][1]+matrixTrain[0][1]
print(matrixTrain)
matrixTest = confusion_matrix(y_test, predictTest)
spamsTest = matrixTest[0][0]/matrixTest[0][0]+matrixTest[1][0]
hamsTest = matrixTest[1][1]/matrixTest[1][1]+matrixTest[0][1]

[[ 577   21]
 [  10 3849]]


In [103]:
# Ajuste manual del parámetro de suavizado
# Modificar algo, esta copiado y pegado !!!!!!!!!!!!!!!!!!!!!!!!!!
from sklearn.model_selection import GridSearchCV
param_grid_nb = {'alpha': [0.0001,0.001, 0.01,0.1, 1, 10,100,200]}
grid_nb = GridSearchCV(MultinomialNB(), param_grid_nb, cv=5)
grid_nb.fit(X_train, y_train)
print("Mejor parámetro: ", grid_nb.best_params_)
print("Rendimiento de MultonomialNB en validación cruzada, con el mejor parámetro: {:.2f}".format(grid_nb.best_score_))

Mejor parámetro:  {'alpha': 0.01}
Rendimiento de MultonomialNB en validación cruzada, con el mejor parámetro: 0.98


In [104]:
# Vectorización con `min_df` y `stop_words`
# Modificar algo, esta copiado y pegado !!!!!!!!!!!!!!!!!!!!!!!!!!
vect2 = CountVectorizer(min_df=80, stop_words="english").fit(text_train)
X2_train = vect2.transform(text_train)
print("Número de términos en el vocabulario original: {}".format(len(vect.get_feature_names())))
feature_names2 = vect2.get_feature_names()
print("Número de términos en el vocabulario con stop words y min_df: {}".format(len(feature_names2)))

Número de términos en el vocabulario original: 8713
Número de términos en el vocabulario con stop words y min_df: 50


/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
